In [ ]:
import arcpy
arcpy.env.overwriteOutput = True

# -------------------
# USER INPUTS
# -------------------
workspace = r"C:\GIS\GoshawkThresholdZones.gdb"
arcpy.env.workspace = workspace
# H:\DataTransfer\GoshawkThresholdZones.gdb\goshawk_nests_usfs
nests    = "goshawk_nests_usfs"
hab_high = "goshawk_ecobject_cwhr_habmodel_high"
hab_mod  = "goshawk_ecobject_cwhr_habmodel_moderate"
out_name = "TRPA_Goshawk_Threshold_Zone"
out_fc   = workspace + "\\" + out_name

target_total_acres = 500.0
buffer_dist = "0.25 Miles"

# Make “no holes” strict: remove ALL interior holes/parts by setting a huge threshold.
# Units are square units of the feature class coordinate system.
HOLE_AREA_THRESHOLD = 10**18

scratch = arcpy.env.scratchGDB  # disk-backed temp (reliable)

# -------------------
# HELPERS
# -------------------
def assert_exists(path, label):
    if not arcpy.Exists(path):
        raise RuntimeError(f"{label} was not created: {path}")

def add_acres(fc, field_name="Acres"):
    if field_name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.management.AddField(fc, field_name, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        fc, [[field_name, "AREA_GEODESIC"]],
        area_unit="ACRES"
    )

def sum_field(fc, field_name):
    return sum((row[0] or 0.0) for row in arcpy.da.SearchCursor(fc, [field_name]))

def ensure_field(fc, name, ftype):
    if name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.management.AddField(fc, name, ftype)

def cursor_order_by(fc, fields, order_by_field):
    # Best-effort ORDER BY (works in file gdb). Falls back if not supported.
    try:
        return arcpy.da.SearchCursor(fc, fields, sql_clause=(None, f"ORDER BY {order_by_field} ASC"))
    except Exception:
        return arcpy.da.SearchCursor(fc, fields)

def chunked_in_clause(field_name, values, chunk_size=900):
    values = list(values)
    for i in range(0, len(values), chunk_size):
        chunk = values[i:i+chunk_size]
        yield f"{field_name} IN ({','.join(map(str, chunk))})"

def copy_selected_oids(work_fc, oids, out_path):
    """Copy selected OIDs from work_fc to out_path."""
    if not oids:
        return None
    lyr = "sel_lyr"
    arcpy.management.MakeFeatureLayer(work_fc, lyr)
    arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")

    oid_f = arcpy.Describe(work_fc).OIDFieldName
    for clause in chunked_in_clause(oid_f, oids):
        arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", clause)

    arcpy.management.CopyFeatures(lyr, out_path)
    assert_exists(out_path, "Selection copy")
    return out_path

def dissolve_parts(parts, out_path):
    arcpy.management.Merge([p for p in parts if p and arcpy.Exists(p)], out_path + "_m")
    assert_exists(out_path + "_m", "Merged")
    arcpy.management.Dissolve(out_path + "_m", out_path)
    assert_exists(out_path, "Dissolved")
    return out_path

def fill_holes(poly_fc, out_path):
    # Removes ALL interior holes/parts (and any tiny stray bits) by setting threshold huge.
    arcpy.management.EliminatePolygonPart(
        in_features=poly_fc,
        out_feature_class=out_path,
        condition="AREA",
        part_area=HOLE_AREA_THRESHOLD,
        part_option="ANY"
    )
    assert_exists(out_path, "Hole-filled polygon")
    return out_path

# -------------------
# OUTPUT FC (one polygon per nest)
# -------------------
sr = arcpy.Describe(hab_high).spatialReference
if arcpy.Exists(out_fc):
    arcpy.management.Delete(out_fc)

arcpy.management.CreateFeatureclass(workspace, out_name, "POLYGON", spatial_reference=sr)
for fn, ft in [
    ("NestOID", "LONG"),
    ("BufAc", "DOUBLE"),
    ("HighAc", "DOUBLE"),
    ("ModAc", "DOUBLE"),
    ("TotalAc", "DOUBLE"),
]:
    arcpy.management.AddField(out_fc, fn, ft)

# -------------------
# PER-NEST CONTIGUOUS GROWTH
# -------------------
nest_oid_field = arcpy.Describe(nests).OIDFieldName
nest_oids = [r[0] for r in arcpy.da.SearchCursor(nests, [nest_oid_field])]
print(f"Nests found: {len(nest_oids)}")

for idx, nest_oid in enumerate(nest_oids, start=1):
    print(f"\n[{idx}/{len(nest_oids)}] Nest OID {nest_oid}")

    # Single nest layer
    nest_lyr = "nest_lyr"
    arcpy.management.MakeFeatureLayer(nests, nest_lyr, f"{nest_oid_field} = {nest_oid}")

    # Required buffer
    buf_fc = scratch + fr"\buf_{nest_oid}"
    arcpy.analysis.Buffer(nest_lyr, buf_fc, buffer_dist, dissolve_option="ALL")
    assert_exists(buf_fc, "Buffer")

    add_acres(buf_fc, "Acres")
    buf_ac = sum_field(buf_fc, "Acres")
    need_out = target_total_acres - buf_ac

    # If buffer already >= 500, output buffer (filled)
    if need_out <= 0:
        final_fc = scratch + fr"\final_{nest_oid}"
        fill_holes(buf_fc, final_fc)
        add_acres(final_fc, "Acres")
        total_ac = sum_field(final_fc, "Acres")

        with arcpy.da.InsertCursor(out_fc, ["SHAPE@", "NestOID", "BufAc", "HighAc", "ModAc", "TotalAc"]) as ic:
            for (geom,) in arcpy.da.SearchCursor(final_fc, ["SHAPE@"]):
                ic.insertRow([geom, nest_oid, float(buf_ac), 0.0, 0.0, float(total_ac)])

        print(f"  Buffer-only Total: {total_ac:.2f} acres (>= 500)")
        continue

    # Erase buffer from habitat (prevents double counting where habitat overlaps buffer)
    high_out = scratch + fr"\high_out_{nest_oid}"
    mod_out  = scratch + fr"\mod_out_{nest_oid}"
    arcpy.analysis.Erase(hab_high, buf_fc, high_out)
    arcpy.analysis.Erase(hab_mod,  buf_fc, mod_out)
    assert_exists(high_out, "High outside")
    assert_exists(mod_out, "Mod outside")

    # Work copies that we can add fields to + compute Near once
    high_work = scratch + fr"\high_work_{nest_oid}"
    mod_work  = scratch + fr"\mod_work_{nest_oid}"
    arcpy.management.CopyFeatures(high_out, high_work)
    arcpy.management.CopyFeatures(mod_out,  mod_work)
    assert_exists(high_work, "High work")
    assert_exists(mod_work,  "Mod work")

    # Add Acres and NEAR_DIST (to the nest point) for compact growth ordering
    add_acres(high_work, "Acres")
    add_acres(mod_work,  "Acres")
    arcpy.analysis.Near(high_work, nest_lyr)  # adds NEAR_DIST
    arcpy.analysis.Near(mod_work,  nest_lyr)

    oid_high = arcpy.Describe(high_work).OIDFieldName
    oid_mod  = arcpy.Describe(mod_work).OIDFieldName

    # Layers for spatial selection
    high_lyr = "high_lyr"
    mod_lyr  = "mod_lyr"
    arcpy.management.MakeFeatureLayer(high_work, high_lyr)
    arcpy.management.MakeFeatureLayer(mod_work,  mod_lyr)

    # Maintain current contiguous footprint (starts at buffer)
    current_fc = scratch + fr"\current_{nest_oid}"
    arcpy.management.CopyFeatures(buf_fc, current_fc)

    selected_high_oids = set()
    selected_mod_oids  = set()
    high_added = 0.0
    mod_added  = 0.0

    # ---- Contiguous growth loop
    # Strategy:
    #   - Add HIGH polygons that touch the current footprint (contiguous), closest-to-nest first
    #   - If still short, add MOD polygons that touch the updated footprint, closest first
    #   - Update footprint by dissolving buffer + selected
    #
    # Note: This guarantees contiguity to the buffer. It won’t “jump” to isolated patches.
    #
    max_iters = 5000  # safety
    it = 0
    while (high_added + mod_added) < need_out and it < max_iters:
        it += 1
        made_progress = False

        # --- Try HIGH first
        if (high_added + mod_added) < need_out:
            arcpy.management.SelectLayerByAttribute(high_lyr, "CLEAR_SELECTION")
            arcpy.management.SelectLayerByLocation(
                in_layer=high_lyr,
                overlap_type="INTERSECT",
                select_features=current_fc,
                selection_type="NEW_SELECTION"
            )

            # Pull candidates in near order, add until we meet need or run out
            # Filter out already selected.
            where = None
            if selected_high_oids:
                # Exclude previously selected
                where = f"{oid_high} NOT IN ({','.join(map(str, sorted(selected_high_oids)))})"

            candidates = []
            if where:
                with cursor_order_by(high_work, [oid_high, "Acres", "NEAR_DIST"], "NEAR_DIST") as cur:
                    for oid, acres, nd in cur:
                        # Keep only those currently selected by location AND not selected before
                        # We’ll check membership via a quick selection layer test:
                        pass

                # Faster: use the selection set directly from the layer
                # and then sort by NEAR_DIST from the work FC.
            # Get selected OIDs from the layer
            sel_oids = set()
            with arcpy.da.SearchCursor(high_lyr, [oid_high]) as sc:
                for (o,) in sc:
                    if o not in selected_high_oids:
                        sel_oids.add(o)

            if sel_oids:
                # Sort selected candidates by NEAR_DIST asc
                rows = []
                where_sel = f"{oid_high} IN ({','.join(map(str, sorted(sel_oids)))})"
                with arcpy.da.SearchCursor(high_work, [oid_high, "Acres", "NEAR_DIST"], where_clause=where_sel) as cur:
                    for o, a, nd in cur:
                        rows.append((o, float(a or 0.0), float(nd if nd is not None else 1e18)))
                rows.sort(key=lambda x: x[2])

                for o, a, _ in rows:
                    if a <= 0:
                        continue
                    selected_high_oids.add(o)
                    high_added += a
                    made_progress = True
                    if (high_added + mod_added) >= need_out:
                        break

        # --- If still short, try MOD (must touch current footprint)
        if (high_added + mod_added) < need_out:
            arcpy.management.SelectLayerByAttribute(mod_lyr, "CLEAR_SELECTION")
            arcpy.management.SelectLayerByLocation(
                in_layer=mod_lyr,
                overlap_type="INTERSECT",
                select_features=current_fc,
                selection_type="NEW_SELECTION"
            )

            sel_oids = set()
            with arcpy.da.SearchCursor(mod_lyr, [oid_mod]) as sc:
                for (o,) in sc:
                    if o not in selected_mod_oids:
                        sel_oids.add(o)

            if sel_oids:
                rows = []
                where_sel = f"{oid_mod} IN ({','.join(map(str, sorted(sel_oids)))})"
                with arcpy.da.SearchCursor(mod_work, [oid_mod, "Acres", "NEAR_DIST"], where_clause=where_sel) as cur:
                    for o, a, nd in cur:
                        rows.append((o, float(a or 0.0), float(nd if nd is not None else 1e18)))
                rows.sort(key=lambda x: x[2])

                for o, a, _ in rows:
                    if a <= 0:
                        continue
                    selected_mod_oids.add(o)
                    mod_added += a
                    made_progress = True
                    if (high_added + mod_added) >= need_out:
                        break

        # If we couldn’t add anything contiguous, stop (no way to reach 500 contiguously)
        if not made_progress:
            break

        # Update current footprint (buffer + selected), dissolved
        parts = [buf_fc]
        if selected_high_oids:
            tmp_high_sel = scratch + fr"\tmp_high_sel_{nest_oid}"
            copy_selected_oids(high_work, sorted(selected_high_oids), tmp_high_sel)
            parts.append(tmp_high_sel)
        if selected_mod_oids:
            tmp_mod_sel = scratch + fr"\tmp_mod_sel_{nest_oid}"
            copy_selected_oids(mod_work, sorted(selected_mod_oids), tmp_mod_sel)
            parts.append(tmp_mod_sel)

        dissolve_parts(parts, current_fc)

    # Build final polygon from buffer + selected contiguous habitat
    parts = [buf_fc]
    if selected_high_oids:
        high_sel = scratch + fr"\high_sel_{nest_oid}"
        copy_selected_oids(high_work, sorted(selected_high_oids), high_sel)
        parts.append(high_sel)
    if selected_mod_oids:
        mod_sel = scratch + fr"\mod_sel_{nest_oid}"
        copy_selected_oids(mod_work, sorted(selected_mod_oids), mod_sel)
        parts.append(mod_sel)

    merged = scratch + fr"\merged_{nest_oid}"
    final_diss = scratch + fr"\final_diss_{nest_oid}"
    arcpy.management.Merge(parts, merged)
    arcpy.management.Dissolve(merged, final_diss)
    assert_exists(final_diss, "Final dissolved")

    # Fill holes (no interior rings)
    final_fc = scratch + fr"\final_noholes_{nest_oid}"
    fill_holes(final_diss, final_fc)

    add_acres(final_fc, "Acres")
    total_ac = sum_field(final_fc, "Acres")

    # Write to output
    with arcpy.da.InsertCursor(out_fc, ["SHAPE@", "NestOID", "BufAc", "HighAc", "ModAc", "TotalAc"]) as ic:
        for (geom,) in arcpy.da.SearchCursor(final_fc, ["SHAPE@"]):
            ic.insertRow([geom, nest_oid, float(buf_ac), float(high_added), float(mod_added), float(total_ac)])

    # Reporting
    if (high_added + mod_added) < need_out:
        print(f"  WARNING: Could not reach 500 acres contiguously.")
    print(f"  Buffer: {buf_ac:.2f} | High(add): {high_added:.2f} | Mod(add): {mod_added:.2f} | Total: {total_ac:.2f}")

print(f"\nDone. Output: {out_fc}")


In [ ]:
import os
import uuid
import shutil
import tempfile
import arcpy

arcpy.env.overwriteOutput = True

# -------------------
# USER INPUTS
# -------------------
workspace = r"C:\GIS\GoshawkThresholdZones.gdb"
arcpy.env.workspace = workspace

nests    = "goshawk_nests_usfs"
hab_high = "goshawk_ecobject_cwhr_habmodel_high"
hab_mod  = "goshawk_ecobject_cwhr_habmodel_moderate"

out_name = "TRPA_Goshawk_Threshold_Zone"
out_fc   = os.path.join(workspace, out_name)

target_total_acres = 500.0
buffer_dist = "0.25 Miles"

# AOI radius to limit habitat processing per nest (increase if you can't reach 500 acres)
aoi_dist = "3 Miles"

# Make “no holes” strict by eliminating any interior parts (very large threshold)
HOLE_AREA_THRESHOLD = 10**18

# -------------------
# DEDICATED SCRATCH WORKSPACE (DELETED AT END)
# -------------------
# Option A: system temp (safe default)
scratch_root = os.path.join(tempfile.gettempdir(), "trpa_goshawk_scratch")
# Option B: point to a big local drive folder (recommended)
# scratch_root = r"D:\GIS_SCRATCH\trpa_goshawk"

os.makedirs(scratch_root, exist_ok=True)

scratch_gdb_name = f"scratch_{uuid.uuid4().hex[:8]}.gdb"
scratch_gdb = os.path.join(scratch_root, scratch_gdb_name)

arcpy.management.CreateFileGDB(scratch_root, scratch_gdb_name)
arcpy.env.scratchWorkspace = scratch_gdb

scratch = scratch_gdb  # use this for ALL temp outputs
print(f"Scratch GDB: {scratch_gdb}")

# -------------------
# HELPERS
# -------------------
def assert_exists(path, label):
    if not arcpy.Exists(path):
        raise RuntimeError(f"{label} was not created: {path}")

def safe_delete(path_or_name):
    """Delete dataset or layer if it exists (no-throw)."""
    try:
        if path_or_name and arcpy.Exists(path_or_name):
            arcpy.management.Delete(path_or_name)
    except Exception:
        pass

def add_acres(fc, field_name="Acres"):
    if field_name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.management.AddField(fc, field_name, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        fc, [[field_name, "AREA_GEODESIC"]],
        area_unit="ACRES"
    )

def sum_field(fc, field_name):
    return sum((row[0] or 0.0) for row in arcpy.da.SearchCursor(fc, [field_name]))

def chunked_in_clause(field_name, values, chunk_size=900):
    values = list(values)
    for i in range(0, len(values), chunk_size):
        chunk = values[i:i+chunk_size]
        yield f"{field_name} IN ({','.join(map(str, chunk))})"

def copy_selected_oids(work_fc, oids, out_path, temp_layers=None):
    """Copy selected OIDs from work_fc to out_path."""
    if not oids:
        return None

    lyr = "sel_lyr"
    arcpy.management.MakeFeatureLayer(work_fc, lyr)
    if temp_layers is not None:
        temp_layers.add(lyr)

    arcpy.management.SelectLayerByAttribute(lyr, "CLEAR_SELECTION")

    oid_f = arcpy.Describe(work_fc).OIDFieldName
    for clause in chunked_in_clause(oid_f, oids):
        arcpy.management.SelectLayerByAttribute(lyr, "ADD_TO_SELECTION", clause)

    arcpy.management.CopyFeatures(lyr, out_path)
    assert_exists(out_path, "Selection copy")
    return out_path

def dissolve_parts(parts, out_path):
    """Merge then dissolve parts into out_path, deleting merge intermediate."""
    merge_out = out_path + "_m"
    arcpy.management.Merge([p for p in parts if p and arcpy.Exists(p)], merge_out)
    assert_exists(merge_out, "Merged")
    arcpy.management.Dissolve(merge_out, out_path)
    assert_exists(out_path, "Dissolved")
    safe_delete(merge_out)  # delete immediately
    return out_path

def fill_holes(poly_fc, out_path):
    arcpy.management.EliminatePolygonPart(
        in_features=poly_fc,
        out_feature_class=out_path,
        condition="AREA",
        part_area=HOLE_AREA_THRESHOLD,
        part_option="ANY"
    )
    assert_exists(out_path, "Hole-filled polygon")
    return out_path

def make_aoi(nest_layer, out_fc, aoi_distance):
    arcpy.analysis.Buffer(nest_layer, out_fc, aoi_distance, dissolve_option="ALL")
    assert_exists(out_fc, "AOI buffer")
    return out_fc

# -------------------
# OUTPUT FC (one polygon per nest)
# -------------------
sr = arcpy.Describe(hab_high).spatialReference
if arcpy.Exists(out_fc):
    arcpy.management.Delete(out_fc)

arcpy.management.CreateFeatureclass(workspace, out_name, "POLYGON", spatial_reference=sr)
for fn, ft in [
    ("NestOID", "LONG"),
    ("BufAc", "DOUBLE"),
    ("HighAc", "DOUBLE"),
    ("ModAc", "DOUBLE"),
    ("TotalAc", "DOUBLE"),
]:
    arcpy.management.AddField(out_fc, fn, ft)

# -------------------
# PER-NEST CONTIGUOUS GROWTH
# -------------------
nest_oid_field = arcpy.Describe(nests).OIDFieldName
nest_oids = [r[0] for r in arcpy.da.SearchCursor(nests, [nest_oid_field])]
print(f"Nests found: {len(nest_oids)}")

for idx, nest_oid in enumerate(nest_oids, start=1):
    print(f"\n[{idx}/{len(nest_oids)}] Nest OID {nest_oid}")

    # Track temps created for this nest so we can delete them no matter what
    temps = set()        # feature classes, tables, etc.
    temp_layers = set()  # layer names

    def T(path):
        """Register a temp dataset path and return it."""
        temps.add(path)
        return path

    def L(name):
        """Register a temp layer name and return it."""
        temp_layers.add(name)
        return name

    try:
        # Single nest layer
        nest_lyr = L("nest_lyr")
        arcpy.management.MakeFeatureLayer(nests, nest_lyr, f"{nest_oid_field} = {nest_oid}")

        # Required buffer
        buf_fc = T(os.path.join(scratch, f"buf_{nest_oid}"))
        arcpy.analysis.Buffer(nest_lyr, buf_fc, buffer_dist, dissolve_option="ALL")
        assert_exists(buf_fc, "Buffer")

        add_acres(buf_fc, "Acres")
        buf_ac = sum_field(buf_fc, "Acres")
        need_out = target_total_acres - buf_ac

        # If buffer already >= 500, output buffer (filled)
        if need_out <= 0:
            final_fc = T(os.path.join(scratch, f"final_{nest_oid}"))
            fill_holes(buf_fc, final_fc)
            add_acres(final_fc, "Acres")
            total_ac = sum_field(final_fc, "Acres")

            with arcpy.da.InsertCursor(out_fc, ["SHAPE@", "NestOID", "BufAc", "HighAc", "ModAc", "TotalAc"]) as ic:
                for (geom,) in arcpy.da.SearchCursor(final_fc, ["SHAPE@"]):
                    ic.insertRow([geom, nest_oid, float(buf_ac), 0.0, 0.0, float(total_ac)])

            print(f"  Buffer-only Total: {total_ac:.2f} acres (>= 500)")
            continue

        # ---- AOI clip first (massive disk/time saver) ----
        aoi_fc = T(os.path.join(scratch, f"aoi_{nest_oid}"))
        make_aoi(nest_lyr, aoi_fc, aoi_dist)

        high_clip = T(os.path.join(scratch, f"high_clip_{nest_oid}"))
        mod_clip  = T(os.path.join(scratch, f"mod_clip_{nest_oid}"))
        arcpy.analysis.Clip(hab_high, aoi_fc, high_clip)
        arcpy.analysis.Clip(hab_mod,  aoi_fc, mod_clip)
        assert_exists(high_clip, "High clip")
        assert_exists(mod_clip,  "Mod clip")

        # ---- Now erase buffer from the *clipped* habitat ----
        high_out = T(os.path.join(scratch, f"high_out_{nest_oid}"))
        mod_out  = T(os.path.join(scratch, f"mod_out_{nest_oid}"))
        arcpy.analysis.Erase(high_clip, buf_fc, high_out)
        arcpy.analysis.Erase(mod_clip,  buf_fc, mod_out)
        assert_exists(high_out, "High outside")
        assert_exists(mod_out,  "Mod outside")

        # Work copies
        high_work = T(os.path.join(scratch, f"high_work_{nest_oid}"))
        mod_work  = T(os.path.join(scratch, f"mod_work_{nest_oid}"))
        arcpy.management.CopyFeatures(high_out, high_work)
        arcpy.management.CopyFeatures(mod_out,  mod_work)
        assert_exists(high_work, "High work")
        assert_exists(mod_work,  "Mod work")

        # Add Acres and NEAR_DIST
        add_acres(high_work, "Acres")
        add_acres(mod_work,  "Acres")
        arcpy.analysis.Near(high_work, nest_lyr)  # adds NEAR_DIST
        arcpy.analysis.Near(mod_work,  nest_lyr)

        oid_high = arcpy.Describe(high_work).OIDFieldName
        oid_mod  = arcpy.Describe(mod_work).OIDFieldName

        # Layers for spatial selection
        high_lyr = L("high_lyr")
        mod_lyr  = L("mod_lyr")
        arcpy.management.MakeFeatureLayer(high_work, high_lyr)
        arcpy.management.MakeFeatureLayer(mod_work,  mod_lyr)

        # Maintain current contiguous footprint (starts at buffer)
        current_fc = T(os.path.join(scratch, f"current_{nest_oid}"))
        arcpy.management.CopyFeatures(buf_fc, current_fc)

        selected_high_oids = set()
        selected_mod_oids  = set()
        high_added = 0.0
        mod_added  = 0.0

        max_iters = 5000
        it = 0
        while (high_added + mod_added) < need_out and it < max_iters:
            it += 1
            made_progress = False

            # --- HIGH first
            if (high_added + mod_added) < need_out:
                arcpy.management.SelectLayerByAttribute(high_lyr, "CLEAR_SELECTION")
                arcpy.management.SelectLayerByLocation(
                    in_layer=high_lyr,
                    overlap_type="INTERSECT",
                    select_features=current_fc,
                    selection_type="NEW_SELECTION"
                )

                sel_oids = set()
                with arcpy.da.SearchCursor(high_lyr, [oid_high]) as sc:
                    for (o,) in sc:
                        if o not in selected_high_oids:
                            sel_oids.add(o)

                if sel_oids:
                    rows = []
                    where_sel = f"{oid_high} IN ({','.join(map(str, sorted(sel_oids)))})"
                    with arcpy.da.SearchCursor(high_work, [oid_high, "Acres", "NEAR_DIST"], where_clause=where_sel) as cur:
                        for o, a, nd in cur:
                            rows.append((o, float(a or 0.0), float(nd if nd is not None else 1e18)))
                    rows.sort(key=lambda x: x[2])

                    for o, a, _ in rows:
                        if a <= 0:
                            continue
                        selected_high_oids.add(o)
                        high_added += a
                        made_progress = True
                        if (high_added + mod_added) >= need_out:
                            break

            # --- MOD next
            if (high_added + mod_added) < need_out:
                arcpy.management.SelectLayerByAttribute(mod_lyr, "CLEAR_SELECTION")
                arcpy.management.SelectLayerByLocation(
                    in_layer=mod_lyr,
                    overlap_type="INTERSECT",
                    select_features=current_fc,
                    selection_type="NEW_SELECTION"
                )

                sel_oids = set()
                with arcpy.da.SearchCursor(mod_lyr, [oid_mod]) as sc:
                    for (o,) in sc:
                        if o not in selected_mod_oids:
                            sel_oids.add(o)

                if sel_oids:
                    rows = []
                    where_sel = f"{oid_mod} IN ({','.join(map(str, sorted(sel_oids)))})"
                    with arcpy.da.SearchCursor(mod_work, [oid_mod, "Acres", "NEAR_DIST"], where_clause=where_sel) as cur:
                        for o, a, nd in cur:
                            rows.append((o, float(a or 0.0), float(nd if nd is not None else 1e18)))
                    rows.sort(key=lambda x: x[2])

                    for o, a, _ in rows:
                        if a <= 0:
                            continue
                        selected_mod_oids.add(o)
                        mod_added += a
                        made_progress = True
                        if (high_added + mod_added) >= need_out:
                            break

            if not made_progress:
                break

            # Update current footprint (buffer + selected), dissolved
            parts = [buf_fc]
            tmp_high_sel = None
            tmp_mod_sel = None

            if selected_high_oids:
                tmp_high_sel = T(os.path.join(scratch, f"tmp_high_sel_{nest_oid}"))
                copy_selected_oids(high_work, sorted(selected_high_oids), tmp_high_sel, temp_layers=temp_layers)
                parts.append(tmp_high_sel)

            if selected_mod_oids:
                tmp_mod_sel = T(os.path.join(scratch, f"tmp_mod_sel_{nest_oid}"))
                copy_selected_oids(mod_work, sorted(selected_mod_oids), tmp_mod_sel, temp_layers=temp_layers)
                parts.append(tmp_mod_sel)

            dissolve_parts(parts, current_fc)

            # IMPORTANT: delete the tmp selections created *inside* the while loop immediately
            safe_delete(tmp_high_sel)
            safe_delete(tmp_mod_sel)
            if tmp_high_sel: temps.discard(tmp_high_sel)
            if tmp_mod_sel:  temps.discard(tmp_mod_sel)

        # Build final polygon from buffer + selected contiguous habitat
        parts = [buf_fc]
        high_sel = None
        mod_sel = None

        if selected_high_oids:
            high_sel = T(os.path.join(scratch, f"high_sel_{nest_oid}"))
            copy_selected_oids(high_work, sorted(selected_high_oids), high_sel, temp_layers=temp_layers)
            parts.append(high_sel)

        if selected_mod_oids:
            mod_sel = T(os.path.join(scratch, f"mod_sel_{nest_oid}"))
            copy_selected_oids(mod_work, sorted(selected_mod_oids), mod_sel, temp_layers=temp_layers)
            parts.append(mod_sel)

        merged = T(os.path.join(scratch, f"merged_{nest_oid}"))
        final_diss = T(os.path.join(scratch, f"final_diss_{nest_oid}"))
        arcpy.management.Merge(parts, merged)
        arcpy.management.Dissolve(merged, final_diss)
        assert_exists(final_diss, "Final dissolved")

        # Fill holes
        final_fc = T(os.path.join(scratch, f"final_noholes_{nest_oid}"))
        fill_holes(final_diss, final_fc)

        add_acres(final_fc, "Acres")
        total_ac = sum_field(final_fc, "Acres")

        # Write to output
        with arcpy.da.InsertCursor(out_fc, ["SHAPE@", "NestOID", "BufAc", "HighAc", "ModAc", "TotalAc"]) as ic:
            for (geom,) in arcpy.da.SearchCursor(final_fc, ["SHAPE@"]):
                ic.insertRow([geom, nest_oid, float(buf_ac), float(high_added), float(mod_added), float(total_ac)])

        # Reporting
        if (high_added + mod_added) < need_out:
            print(f"  WARNING: Could not reach 500 acres contiguously within AOI ({aoi_dist}). Consider increasing aoi_dist.")
        print(f"  Buffer: {buf_ac:.2f} | High(add): {high_added:.2f} | Mod(add): {mod_added:.2f} | Total: {total_ac:.2f}")

    finally:
        # ---- Cleanup per nest (disk + memory) ----
        # Delete temp layers first
        for lyr in list(temp_layers):
            safe_delete(lyr)

        # Delete temp datasets
        for ds in list(temps):
            safe_delete(ds)

        # Clear workspace cache to release locks and free scratch items
        try:
            arcpy.ClearWorkspaceCache_management()
        except Exception:
            pass

print(f"\nDone. Output: {out_fc}")

# -------------------
# DELETE SCRATCH GDB AT END OF RUN
# -------------------
try:
    arcpy.ClearWorkspaceCache_management()
except Exception:
    pass

try:
    shutil.rmtree(scratch_gdb, ignore_errors=True)
    print(f"Deleted scratch: {scratch_gdb}")
except Exception as e:
    print(f"Could not delete scratch GDB (locks?): {scratch_gdb}\n{e}")


Scratch GDB: C:\Users\mbindl\AppData\Local\Temp\2\trpa_goshawk_scratch\scratch_3aa04335.gdb
Nests found: 136

[1/136] Nest OID 1
  Buffer: 125.55 | High(add): 220.12 | Mod(add): 157.85 | Total: 546.78

[2/136] Nest OID 2
  Buffer: 125.55 | High(add): 236.76 | Mod(add): 152.29 | Total: 544.62

[3/136] Nest OID 3
  Buffer: 125.56 | High(add): 252.68 | Mod(add): 122.57 | Total: 539.67

[4/136] Nest OID 4
  Buffer: 125.53 | High(add): 200.63 | Mod(add): 185.41 | Total: 549.60

[5/136] Nest OID 5
  Buffer: 125.53 | High(add): 246.17 | Mod(add): 130.03 | Total: 539.78

[6/136] Nest OID 6
  Buffer: 125.52 | High(add): 105.85 | Mod(add): 254.35 | Total: 526.53

[7/136] Nest OID 7
  Buffer: 125.54 | High(add): 79.25 | Mod(add): 298.18 | Total: 540.39

[8/136] Nest OID 8
  Buffer: 125.54 | High(add): 298.90 | Mod(add): 76.31 | Total: 515.06

[9/136] Nest OID 9
  Buffer: 125.53 | High(add): 280.15 | Mod(add): 95.20 | Total: 524.21

[10/136] Nest OID 10
  Buffer: 125.53 | High(add): 315.88 | Mod(a